## Group Normalization

Ref: <a href='https://arxiv.org/abs/1803.08494'>Group Normalization</a>

Group Normalization(GN)은 Layer Normalization에 이어, 배치 사이즈에 영향을 많이 받는 Batch Normalization(BN)의 한계를 극복하기 위해 제안된 정규화 기법이다. 피쳐들을 그룹으로 정규화하는 방식을 취하며, 이는 <a href='https://en.wikipedia.org/wiki/Scale-invariant_feature_transform'>SIFT</a>나 <a href='https://en.wikipedia.org/wiki/Histogram_of_oriented_gradients'>HOG</a>와 같은 고전적인 특징 추출 방법에서 특징을 추출할 때 지역적 패치 혹은 블록(Block) 단위로 정규화가 되는 것에서 고안되었다.

GN은 채널 축을 그룹으로 나누어 각 그룹 내에서 평균과 분산을 계산하여 정규화한다. 이는 입력 채널을 G개의 그룹으로 나누어 각 그룹에 대해 정규화하는 방식으로 배치 크기에 관계없이 항상 안정적인 정규화 효과를 발휘한다.

입력 $X\in \mathbb{R}^{N\times C\times H\times W}$와 그룹 수 $G$에 대해 다음과 같이 계산된다.

1. 각 그룹 내 평균 $\mu_g$와 분산 $\sigma^2_g$ 계산
$$
\mu_g = {1\over C/G \cdot H \cdot W}\Sigma_{c\in\mathcal{G}_g}\Sigma^{H-1}_{h=0}\Sigma^{W-1}_{w=0}X_{n,c,h,w}
$$

$$
\sigma^2_g = {1\over C/G \cdot H \cdot W}\Sigma_{c\in\mathcal{G}_g}\Sigma^{H-1}_{h=0}\Sigma^{W-1}_{w=0}(X_{n,c,h,w}-\mu_g)^2
$$

2. 정규화

$$
\hat{X} = {X_{n,c,h,w}-\mu_g\over \sqrt{\sigma^2_g + \epsilon}}
$$

3. 스케일링 및 쉬프트
$$
GN(X) = \gamma {X_{n,c,h,w}-\mu_g\over \sqrt{\sigma^2_g + \epsilon}} + \beta
$$

In [8]:
import torch
from torch import nn

In [13]:
class GroupNorm(nn.Module):
    '''
    groups: 피쳐를 나누는 그룹 수
    channels: 채널 수
    eps: 엡실론
    affine: 스케일링 및 시프트 적용 여부 불리언
    '''
    
    def __init__(self, groups: int, channels: int, *,
                 eps: float = 1e-5, affine: bool = True):
        super().__init__()
        
        assert channels % groups == 0, "Number of channels should be evenly divisible by the number of groups"
        self.groups = groups
        self.channels = channels
        self.eps = eps
        self.affine = affine
        
        if self.affine:
            self.gamma = nn.Parameter(torch.ones(channels))
            self.beta = nn.Parameter(torch.zeros(channels))
        
    def forward(self, x: torch.Tensor):
        # x.shape = [batch_size, channels, *]
        x_shape = x.shape
        
        batch_size = x_shape[0]
        assert self.channels == x_shape[1]
        
        # reshape into [batch_size, groups, n]
        x = x.view(batch_size, self.groups, -1)
        
        # calculate mean and variance across last dimension
        mean = x.mean(dim=[-1], keepdim=True)
        mean_x2 = (x ** 2).mean(dim=[-1], keepdim=True)
        
        var = mean_x2 - (mean ** 2)
        
        # normalize
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        
        if self.affine:
            # channel-wise
            x_norm = x_norm.view(batch_size, self.channels, -1)
            x_norm = self.gamma.view(1,-1,1) * x_norm + self.beta.view(1,-1,1)
            
        x_norm = x_norm.view(x_shape) # to original shape
        
        return x_norm

In [14]:
def _test():
    x = torch.randn([2,6,2,4]) # B C H W
    print(f'original input: {x})')
    
    gn = GroupNorm(2,6)
    x = gn(x)
    
    print(f'normalized input: {x}')
    print(f'{gn.gamma.shape = }')

In [15]:
if __name__ == '__main__':
    _test()

original input: tensor([[[[-3.1875e-01, -2.5255e+00, -1.9157e+00,  1.4343e-01],
          [ 1.0294e+00,  1.4238e+00, -9.5062e-01, -1.2318e+00]],

         [[ 9.9617e-01, -1.5139e+00,  8.5035e-01,  5.4831e-01],
          [-1.0538e+00,  6.6934e-02,  3.2912e-01,  1.8479e+00]],

         [[-1.5556e-03, -1.0585e-01, -1.2116e+00,  1.1413e+00],
          [-3.0255e-01, -1.6866e-01,  1.1033e+00,  1.8413e+00]],

         [[ 1.3062e+00, -1.3857e+00, -1.3622e+00,  2.0083e-02],
          [-1.1508e+00, -1.8117e+00, -1.7821e-01,  5.5125e-01]],

         [[ 6.9868e-01,  1.3340e+00, -1.9481e+00,  4.0271e-01],
          [-1.1449e-01,  1.0895e+00,  2.4716e-01, -9.2342e-01]],

         [[-6.5360e-02, -4.9357e-01,  1.0296e+00, -8.0111e-01],
          [ 1.3396e+00, -6.0696e-01,  1.2484e+00,  2.1670e-01]]],


        [[[-1.2001e-01, -5.0902e-02, -7.1455e-01,  3.6899e-01],
          [ 3.1756e-01,  2.8267e-01, -2.1198e-01,  1.8537e-01]],

         [[-5.1675e-01, -6.5196e-01,  7.1314e-01, -2.1672e-01],
        